
Para variables numéricas:

ANOVA (F-Statistic y p-valor): Para verificar si hay diferencias significativas en la media de la variable numérica entre las 7 categorías.

Gini impurity: Utilizando un árbol de decisión para ver qué tan bien la variable segmenta la variable objetivo.

Kruskal-Wallis: Alternativa no paramétrica a ANOVA cuando la normalidad no está garantizada.

Para variables categóricas:

Chi-cuadrado (χ² y p-valor): Para medir independencia entre la variable categórica y la variable objetivo.

IV (Information Value): Para medir la capacidad de la variable para discriminar entre categorías.

Entropía de Shannon: Para evaluar la incertidumbre de la variable respecto a la variable objetivo.

Cramer’s V: Para medir la fuerza de asociación entre la variable y la variable objetivo.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, chi2_contingency, entropy, kruskal
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import chi2
import itertools

def calculate_gini(x, y):
    """Calcula la importancia de Gini usando un árbol de decisión simple."""
    clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
    clf.fit(x.values.reshape(-1, 1), y)
    return clf.feature_importances_[0]

def information_value(x, y):
    """Calcula el Information Value (IV) para una variable categórica."""
    df = pd.DataFrame({'x': x, 'y': y})
    grouped = df.groupby('x')['y'].value_counts(normalize=True).unstack().fillna(0)
    woe = np.log((grouped + 0.0001) / (1 - grouped + 0.0001))  # WoE
    iv = (grouped - (1 - grouped)) * woe
    return iv.sum().sum()

def cramers_v(x, y):
    """Calcula el coeficiente de Cramer para medir la asociación entre variables categóricas."""
    contingency_table = pd.crosstab(x, y)
    chi2_val, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    return np.sqrt(chi2_val / (n * min_dim))

def analyze_features(df, target_column):
    results = []
    y = df[target_column]
    
    # Convertimos la variable objetivo a numérica si es categórica
    if y.dtype == 'object' or y.dtype.name == 'category':
        y = LabelEncoder().fit_transform(y)

    for column in df.columns:
        if column == target_column:
            continue
        
        x = df[column]
        result = {'Variable': column, 'Tipo': x.dtype}
        
        if x.dtype in ['int64', 'float64']:  # Variables numéricas
            groups = [x[y == cat] for cat in np.unique(y)]
            if len(groups) > 1:
                f_stat, p_value = f_oneway(*groups)
                kw_stat, kw_p_value = kruskal(*groups)
                result.update({'ANOVA_F': f_stat, 'ANOVA_p': p_value, 'Kruskal_H': kw_stat, 'Kruskal_p': kw_p_value})
            else:
                result.update({'ANOVA_F': np.nan, 'ANOVA_p': np.nan, 'Kruskal_H': np.nan, 'Kruskal_p': np.nan})
            
            result['Gini'] = calculate_gini(x.dropna(), y[x.notna()])

        else:  # Variables categóricas
            contingency_table = pd.crosstab(x, y)
            chi2_val, p, _, _ = chi2_contingency(contingency_table)
            result.update({'Chi2': chi2_val, 'Chi2_p': p})
            result['IV'] = information_value(x, y)
            result['Entropía'] = entropy(contingency_table.sum(axis=1), base=2)
            result['Cramer_V'] = cramers_v(x, y)

        results.append(result)

    return pd.DataFrame(results)

# Ejemplo de uso
df = pd.read_csv("tu_archivo.csv")  # Carga tu dataset
target_column = "canal_preferido"  # Nombre de la variable objetivo
df_results = analyze_features(df, target_column)

# Mostrar los resultados ordenados por importancia
df_results.sort_values(by=["ANOVA_F", "Chi2", "IV", "Gini", "Cramer_V"], ascending=False, na_position='last')
